# TP3 - Analisador Léxico

Construir um analisador léxico para uma linguagem de query com a qual se podem escrever frases do género:

```sparql
# DBPedia: obras de Chuck Berry
select ?nome ?desc where {
    ?s a dbo:MusicalArtist.
    ?s foaf:name "Chuck Berry"@en .
    ?w dbo:artist ?s.
    ?w foaf:name ?nome.
    ?w dbo:abstract ?desc
} LIMIT 1000

## Tokens 

In [3]:
tokens = [
        ('NEWLINE',r"\n"),
        ('COMENTARIO', r"#.*"),
        ('STRING', r'"[^"]*"'),
        ('SELECT', r"\bselect\b"),
        ('WHERE', r"\bwhere\b"),
        ('LIMIT', r"\bLIMIT\b"),
        ('VAR', r"\?[A-Za-z_][A-Za-z0-9]*"),
        ('PREFIX', r"[A-Za-z_][A-Za-z0-9_]*:[A-Za-z_][A-Za-z0-9_]*"),
        ('IDIOMA', r"@[a-zA-Z\-]+"),
        ('A', r"\ba\b"),
        ('LBRACE', r"\{"),
        ('RBRACE', r"\}"),
        ('PONTO', r"\."),
        ('NUM', r"[0-9]+"),
        ('SKIP', r"[ \t\r]+"),
        ('ERRO', r".")
    ]

## Código

In [4]:
import re

def analisador(c):
    reconhecidos = []
    linha = 1
    regex = '|'.join(f'(?P<{nome}>{pattern})' for nome, pattern in tokens)
    val = re.finditer(regex,c)

    for v in val:
        dic = v.groupdict()
        valor = v.group()
        for k in dic:
            if dic[k]:
                tipo = k
                break
        
        if tipo == 'NEWLINE':          
            #vou considerar que não é suposto mostrar newlines
            linha+=1
        elif tipo not in ('SKIP', 'COMENTARIO'):    #nao faz sentido contar os comentarios ou os espaços
            reconhecidos.append((tipo,valor,linha,v.span()))
    
    return reconhecidos


## Teste

In [5]:
query = """# DBPedia: obras de Chuck Berry
select ?nome ?desc where {
?s a dbo:MusicalArtist.
?s foaf:name "Chuck Berry"@en .
?w dbo:artist ?s.
?w foaf:name ?nome.
?w dbo:abstract ?desc
} LIMIT 1000
"""

for t in analisador(query):
    print (t)

('SELECT', 'select', 2, (32, 38))
('VAR', '?nome', 2, (39, 44))
('VAR', '?desc', 2, (45, 50))
('WHERE', 'where', 2, (51, 56))
('LBRACE', '{', 2, (57, 58))
('VAR', '?s', 3, (59, 61))
('A', 'a', 3, (62, 63))
('PREFIX', 'dbo:MusicalArtist', 3, (64, 81))
('PONTO', '.', 3, (81, 82))
('VAR', '?s', 4, (83, 85))
('PREFIX', 'foaf:name', 4, (86, 95))
('STRING', '"Chuck Berry"', 4, (96, 109))
('IDIOMA', '@en', 4, (109, 112))
('PONTO', '.', 4, (113, 114))
('VAR', '?w', 5, (115, 117))
('PREFIX', 'dbo:artist', 5, (118, 128))
('VAR', '?s', 5, (129, 131))
('PONTO', '.', 5, (131, 132))
('VAR', '?w', 6, (133, 135))
('PREFIX', 'foaf:name', 6, (136, 145))
('VAR', '?nome', 6, (146, 151))
('PONTO', '.', 6, (151, 152))
('VAR', '?w', 7, (153, 155))
('PREFIX', 'dbo:abstract', 7, (156, 168))
('VAR', '?desc', 7, (169, 174))
('RBRACE', '}', 8, (175, 176))
('LIMIT', 'LIMIT', 8, (177, 182))
('NUM', '1000', 8, (183, 187))
